<a href="https://colab.research.google.com/github/GwiHwan-Go/Reproduce_CutMix/blob/main/Experiments/tinyimagenet_resent18_cutmix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200

--2022-05-24 16:56:56--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  22.0MB/s    in 17s     

2022-05-24 16:57:13 (13.9 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]

test  train  val  wnids.txt  words.txt


In [2]:
import os
import torch
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data.dataloader import *
import cv2 # OpenCV


In [3]:
with open('./tiny-imagenet-200/wnids.txt', 'r') as f :
  label_id = f.readlines()
label_id = [i[:-1] for i in label_id]
len(label_id)

200

In [4]:
def parseClasses(file):
    classes = []
    filenames = []
    with open(file) as f:
        lines = f.readlines()
    lines = [x.strip() for x in lines]
    for x in range(0,len(lines)):
        tokens = lines[x].split()
        classes.append(tokens[1])
        filenames.append(tokens[0])
    return filenames,classes
    
class TImgNetDataset(Dataset):
    """Dataset wrapping images and ground truths."""
    
    def __init__(self, img_path, gt_path, class_to_idx=None, transform=None):
        self.img_path = img_path
        self.transform = transform
        self.gt_path = gt_path
        self.class_to_idx = class_to_idx
        self.classidx = []
        self.imgs, self.classnames = parseClasses(gt_path)
        for classname in self.classnames:
            self.classidx.append(self.class_to_idx[classname])

    def __getitem__(self, index):
            """
            Args:
                index (int): Index
            Returns:
                tuple: (image, y) where y is the label of the image.
            """
            img = None
            with open(os.path.join(self.img_path, self.imgs[index]), 'rb') as f:
                img = Image.open(f)
                img = img.convert('RGB')
                if self.transform is not None:
                    img = self.transform(img)
            y = self.classidx[index]
            return img, y

    def __len__(self):
        return len(self.imgs)

In [5]:
batch_size = 128
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,

        ])
test_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            normalize,

        ])

In [6]:
import torchvision.datasets as datasets
import numpy as np

train_set = datasets.ImageFolder("./tiny-imagenet-200/train", transform=train_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

val_set = TImgNetDataset("./tiny-imagenet-200/val/images", "./tiny-imagenet-200/val/val_annotations.txt", class_to_idx=train_loader.dataset.class_to_idx.copy(),
            transform=test_transform)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

test_set = datasets.ImageFolder("./tiny-imagenet-200/test", transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

def cut(W,H,lam):
        
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int64(W * cut_rat)
    cut_h = np.int64(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [7]:
from torch.nn import Linear
from torchvision import models

# resnet50 Model Loading
use_pretrained = False
model = models.resnet18(pretrained=use_pretrained)
## 우리 이미지 사이즈에 맞게 튜닝

model.fc = torch.nn.Linear(512, 200) 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# model = model.to(device)

cuda


In [8]:
import torch.optim as optim
import torch.nn as nn
from tqdm.auto import tqdm, trange

    
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
        wrong_k = batch_size - correct_k
        res.append(wrong_k.mul_(100.0 / batch_size))
    return res
    
def train_model(model, 
                train, 
                valid,
                resume = None,
                n_iters=300, 
                learn_rate=0.1, 
                weight_decay=0, 
                which_method=0
                ):  # Lists to store model's performance information
  po = 0
  top1_errs, top5_errs, train_losses = [], [], []
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=learn_rate, momentum=0.9, weight_decay=weight_decay)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print(f"we are going to use {device}")
###############Checkpoint Zone################
  if resume is not None :
    checkpoint = torch.load(resume) 
    model.load_state_dict(checkpoint['model_state_dict'])
    loss = checkpoint['loss']
    num_epochs = checkpoint['total_epochs'] - checkpoint['epoch']
    top1_errs = checkpoint['top1_err']
    top5_errs = checkpoint['top5_err']
  else :
    num_epochs = n_iters
###############Checkpoint Zone###############
  
  print(f"We still have to go {num_epochs} epochs" )
  for i in trange(num_epochs):

    ##############################learning_decay#####################################
    if (num_epochs-i) < 75 :
      newpo = 3
    elif (num_epochs-i) < 150 :
      newpo = 2
    elif (num_epochs-i) < 225 :
      newpo = 1
    else :
      newpo = 0
    if po != newpo :
      po = newpo
      lr = learn_rate * (0.1 ** po) ##learning_decay
      print(f"lr has decayed to {lr}")
      optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    ##############################learning_decay#####################################

    try :
      for images, labels in tqdm(train) :

        images, labels = images.to(device), labels.to(device)
        model = model.to(device)
        model.train() # For Dropout and Batch Norm layers #

        if which_method and (np.random.rand()<0.5) :

          lam = np.random.beta(1.0, 1.0)
          rand_index = torch.randperm(images.size()[0])
          shuffled_labels = labels[rand_index]

          ##############Do Something On Data###############
          if which_method == 1 : ##cutmix
            bbx1, bby1, bbx2, bby2 = cut(images.shape[2], images.shape[3], lam) # define a box to cut and mix
            images[:, :, bbx1:bbx2, bby1:bby2] = images[rand_index, :, bbx1:bbx2, bby1:bby2] #cut and mix
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.shape[-1] * images.shape[-2]))
          if which_method == 2 : ##mixup
            images = lam * images + (1 - lam) * images[rand_index,:,:]
          if which_method == 3 : ##cutout
            bbx1, bby1, bbx2, bby2 = cut(images.shape[2], images.shape[3], lam) # define a box to cut and mix
            images[:, :, bbx1:bbx2, bby1:bby2] = 0.
          ##############Do Something On Data###############

          #############Forward Pass##############
          out = model(images) 
          #############Forward Pass##############

          ##############Modify Loss Function###############
          if which_method == 1 : ##cutmix
            loss = criterion(out, labels) * lam + criterion(out, shuffled_labels)*(1.0-lam) # compute the total loss
          if which_method == 2 : ##mixup
            loss = criterion(out, labels) * lam + criterion(out, shuffled_labels)*(1.0-lam)
          if which_method == 3 : ##cutout
            loss = criterion(out, labels)
          ##############Modify Loss Function###############

        else : ## if random.rand > 0.5 or which_method == 0
          
          out = model(images)
          loss = criterion(out, labels)

        loss.backward()               # backward pass (compute parameter updates)
        optimizer.step()              # make the updates for each parameter
        optimizer.zero_grad()         # reset the gradients for the next iteration
      
      #####################model Evaluation#############
      sum_top1, sum_top5 = 0,0
      
                
      with torch.no_grad():
        model.eval()
        for images, labels in valid :

          images,labels = images.to(device), labels.to(device)
          
          output = model(images)
          
          top1, top5 = accuracy(output, labels, topk=(1,5))
   
          sum_top1+=top1.item()

          sum_top5+=top5.item()
      
      size = len(valid)
      #####################model Evaluation#############

      ###################save history#####################
      train_losses.append(loss.item())
      top1_errs.append(sum_top1/size)
      top5_errs.append(sum_top5/size)
      ###################save history#####################

      if (i+1) % 10 == 0 :

        PATH=f"./{300-(num_epochs-i)}th_checkpoint.pt"
        torch.save({'total_epochs':num_epochs, 'epoch': i, 'model_state_dict': model.state_dict(), 
        'optimizer_state_dict': optimizer.state_dict(), 'loss': loss, 'top1_err': top1_errs, 'top5_err': top5_errs, 'train_loss':train_losses }, PATH)
        print(f"Successfully saved untill {i} step in {PATH} with model, optimizer, and loss.")

    except :
      PATH=f"./{300-(num_epochs-i)}th_checkpoint.pt"
      torch.save({'total_epochs':num_epochs, 'epoch': i, 'model_state_dict': model.state_dict(), 
      'optimizer_state_dict': optimizer.state_dict(), 'loss': loss, 'top1_err': top1_errs, 'top5_err': top5_errs , 'train_loss':train_losses}, PATH)
      print(f"Successfully saved untill {i} step in {PATH} with model, optimizer, and loss.")
      return [top1_errs,top5_errs]

  PATH = "./trained.pt"
  print("train finished")
  print(f"Successfully saved untill {n_iters} step in {PATH} with model, optimizer, and loss.")
  torch.save({'total_epochs':num_epochs, 'epoch': i, 'model_state_dict': model.state_dict(), 
  'optimizer_state_dict': optimizer.state_dict(), 'loss': loss, 'top1_err': top1_errs, 'top5_err': top5_errs, 'train_loss':train_losses}, PATH)
  return [top1_errs,top5_errs]


In [9]:
import torch
print(torch.cuda.memory_allocated()/1024**2)
import gc
gc.collect()
a = torch.cuda.empty_cache() 
print(torch.cuda.memory_allocated()/1024**2)


0.0
0.0


In [ ]:
##You should run this cell ONLY FIRST TIME!!!
##Otherwise It will OverWrite your Checkpoint file.
print(torch.cuda.memory_allocated()/1024**2)
logs = train_model(model, train_loader, valid_loader, n_iters=300, 
                    learn_rate=0.1, 
                    weight_decay=0, which_method=1) 

#which method 
# 0: nothing, 
# 1: cutmix, 
# 2: mixup 
# 3: cutout

0.0
we are going to use cuda
We still have to go 300 epochs


UnboundLocalError: local variable 'loss' referenced before assignment

In [ ]:
checkpoint = torch.load("./1th_checkpoint.pt")
checkpoint['train_loss']

[5.588218688964844]

In [10]:
## Chek point load 요령 ##
PATH="./181th_checkpoint.pt" ## Where checkpoint saved. ## saved checkpoint

train_model(model,
            train_loader, 
            valid_loader,
            n_iters=300, 
            learn_rate=0.1, 
            resume=PATH,
            weight_decay=0, 
            which_method=1) 

we are going to use cuda
We still have to go 119 epochs


  0%|          | 0/119 [00:00<?, ?it/s]

lr has decayed to 0.0010000000000000002


  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

...

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

train finished
Successfully saved untill 300 step in ./trained.pt with model, optimizer, and loss.


[[90.60522151898734,
  82.45648734177215,
  73.79351265822785,
  67.13805379746836,
  62.539556962025316,
  60.13647151898734,
  60.433148734177216,
  55.00395569620253,
  51.78995253164557,
  51.028481012658226,
  51.157041139240505,
  49.307753164556964,
  49.129746835443036,
  50.14833860759494,
  48.33860759493671,
  49.515427215189874,
  49.06052215189873,
  48.299050632911396,
  47.794699367088604,
  47.78481012658228,
  47.695806962025316,
  48.66495253164557,
  47.81447784810127,
  47.50791139240506,
  47.24090189873418,
  47.80458860759494,
  46.98378164556962,
  46.964003164556964,
  48.21993670886076,
  46.835443037974684,
  46.77610759493671,
  46.97389240506329,
  46.90466772151899,
  46.79588607594937,
  46.133306962025316,
  46.74643987341772,
  46.420094936708864,
  47.09256329113924,
  47.1123417721519,
  46.390427215189874,
  47.171677215189874,
  47.03322784810127,
  47.30023734177215,
  47.359572784810126,
  46.69699367088607,
  47.171677215189874,
  46.973892405063

In [ ]:
!nvidia-smi

Mon May 23 04:58:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:67:00.0 Off |                  N/A |
|100%   75C    P2   342W / 350W |  24055MiB / 24576MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluate

In [ ]:
## If train was succeed, let's evaluate it.

In [11]:
PATH="./trained.pt"
checkpoint = torch.load(PATH) 
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
sum_top1, sum_top5 = 0,0

for images, labels in valid_loader :
    images,labels = images.to(device), labels.to(device)
    output = model(images)
    top1, top5 = accuracy(output, labels, topk=(1,5))
    sum_top1+=top1.item()
    sum_top5+=top5.item()

size = len(valid_loader)

print(sum_top1/size, sum_top5/size)

45.16416139240506 22.44857594936709


In [12]:
checkpoint['top1_err']

[90.60522151898734,
 82.45648734177215,
 73.79351265822785,
 67.13805379746836,
 62.539556962025316,
 60.13647151898734,
 60.433148734177216,
 55.00395569620253,
 51.78995253164557,
 51.028481012658226,
 51.157041139240505,
 49.307753164556964,
 49.129746835443036,
 50.14833860759494,
 48.33860759493671,
 49.515427215189874,
 49.06052215189873,
 48.299050632911396,
 47.794699367088604,
 47.78481012658228,
 47.695806962025316,
 48.66495253164557,
 47.81447784810127,
 47.50791139240506,
 47.24090189873418,
 47.80458860759494,
 46.98378164556962,
 46.964003164556964,
 48.21993670886076,
 46.835443037974684,
 46.77610759493671,
 46.97389240506329,
 46.90466772151899,
 46.79588607594937,
 46.133306962025316,
 46.74643987341772,
 46.420094936708864,
 47.09256329113924,
 47.1123417721519,
 46.390427215189874,
 47.171677215189874,
 47.03322784810127,
 47.30023734177215,
 47.359572784810126,
 46.69699367088607,
 47.171677215189874,
 46.97389240506329,
 47.10245253164557,
 45.72784810126582,
 47

https://github.com/facebookresearch/mixup-cifar10/blob/main/train.py